In [1]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

from pydrake.all import (
    Concatenate,
    ConstantVectorSource,
    DiagramBuilder,
    InverseKinematics,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    PointCloud,
    RigidTransform,
    LogVectorOutput,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
    CollisionFilterDeclaration,
    GeometrySet,
    CollisionFilterManager
)
from pydrake.multibody import inverse_kinematics
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.trajectories import PiecewisePolynomial
from scipy.spatial import KDTree

from manipulation import FindResource
from manipulation.meshcat_utils import AddMeshcatTriad, MeshcatPoseSliders
from manipulation.scenarios import AddIiwa, AddWsg, AddMultibodyTriad, MakeManipulationStation, AddRgbdSensors
from manipulation.station import AddPointClouds, load_scenario, MakeHardwareStation
from manipulation.utils import running_as_notebook, ConfigureParser

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7008


In [5]:
def run_simulation():
    builder = DiagramBuilder()
    time_step = 0.001
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    # Parser(plant).AddModels("model.sdf")
    iiwa = AddIiwa(plant, "with_box_collision")
    wsg = AddWsg(plant, iiwa, welded=True)
    plant.Finalize()

    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    # logger = LogVectorOutput(plant.get_state_output_port(), builder)

    diagram = builder.Build()
    simulator = Simulator(diagram)

    # meshcat.StartRecording()

    context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    # plant.SetPositions(plant_context, [1])
    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)

    def my_callback(context, pose):
        ik = InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(
            gripper_frame,
            [0, 0, 0],
            plant.world_frame(),
            pose.translation(),
            pose.translation(),
        )
        ik.AddOrientationConstraint(
            gripper_frame,
            RotationMatrix(),
            plant.world_frame(),
            pose.rotation(),
            0.0,
        )
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = Solve(ik.prog())
        if result.is_success():
            print("IK success")
        else:
            print("IK failure")
        clear_output(wait=True)

    meshcat.DeleteAddedControls()
    sliders = MeshcatPoseSliders(meshcat)
    sliders.SetPose(
        plant.EvalBodyPoseInWorld(
            plant_context, plant.GetBodyByName("body", wsg)
        )
    )
    sliders.Run(visualizer, context, my_callback)

    # simulator.AdvanceTo(3 if running_as_notebook else 0.1)
    # meshcat.PublishRecording()

# These are the default values that MultibodyPlant picked (on Nov 16, 2022) for
# this system.
run_simulation()

IK failure
